In [1]:
from nipype import Node, Workflow, IdentityInterface, Function, MapNode,SelectFiles, DataSink
from nipype.interfaces import fsl
from nipype.algorithms.modelgen import SpecifyModel
import numpy as np
import os

In [2]:
def pick_first(out_stat):
    return int(out_stat[0])

In [3]:
data = "/media/paradeisios/UUI/fmri/flanker/output/second_level"
output = "/media/paradeisios/UUI/fmri/flanker/output/third_level"
mask_file = "/home/paradeisios/fsl/data/standard/MNI152_T1_2mm_brain_mask.nii.gz"

num_of_contrasts = 6
num_of_subjects = 26
contrast_list = [str(i) for i in range(1,num_of_contrasts+1)]

In [4]:
infosource = Node(IdentityInterface(fields=["contrast_list"]), name="infosource")
infosource.iterables = [("contrast_list", contrast_list)]

In [5]:
templates = {"copes":     "*/fixed_effects/*/fixed_eff_model_{contrast_list}/cope1.nii.gz",
             "varcopes":  "*/fixed_effects/*/fixed_eff_model_{contrast_list}/varcope1.nii.gz"}

selectfiles = Node(SelectFiles(templates),name="selectfiles")
selectfiles.inputs.base_directory = data
selectfiles.inputs.contrast_list=1

In [6]:
substitutions = []
#substitutions += [("_contrast_list_{:d}".format(i+1), contrast_names[i]) for i in range(num_of_contrasts)]

datasink = Node(DataSink(base_directory=output),name="datasink")
datasink.inputs.substitutions = substitutions
datasink.output = output

In [7]:
copemerge = Node(fsl.Merge(),name="copemerge")
copemerge.inputs.dimension = "t"

varcopemerge = Node(fsl.Merge(),name="varcopemerge")
varcopemerge.inputs.dimension = "t"

level2model = Node(fsl.L2Model(), name='l2model')
level2model.inputs.num_copes = num_of_subjects

random_eff_model = Node(fsl.FLAMEO(),name="random_eff_model")
random_eff_model.inputs.run_mode="ols"
random_eff_model.inputs.mask_file= mask_file

cluster_corr = Node(fsl.Cluster(),name="cluster_corr")
cluster_corr.inputs.threshold = 2.3
cluster_corr.inputs.pthreshold = 0.05
cluster_corr.inputs.use_mm = True
cluster_corr.inputs.out_threshold_file = True
cluster_corr.inputs.out_localmax_txt_file = True
cluster_corr.inputs.out_pval_file = True
cluster_corr.inputs.out_index_file = True

randomise = Node(fsl.Randomise(), name="randomise")
randomise.inputs.mask = mask_file
randomise.inputs.tfce = True
randomise.inputs.vox_p_values = True
randomise.inputs.num_perm = 1000

smooth_estimate = Node(fsl.SmoothEstimate(), name="smooth_estimate")
smooth_estimate.inputs.mask_file = mask_file 

num_of_voxels = Node(fsl.ImageStats(), name="num_of_voxels")
num_of_voxels.inputs.in_file = mask_file
num_of_voxels.inputs.op_string = "-V"

In [8]:
random_effects_analysis = Workflow(name="random_effects_analysis",base_dir=output)

random_effects_analysis.connect(infosource,"contrast_list",datasink,"container")
random_effects_analysis.connect(infosource,"contrast_list",selectfiles,"contrast_list")

random_effects_analysis.connect(selectfiles,"copes",copemerge,"in_files")
random_effects_analysis.connect(selectfiles,"varcopes",varcopemerge,"in_files")

random_effects_analysis.connect(copemerge,"merged_file",random_eff_model,"cope_file")
random_effects_analysis.connect(varcopemerge,"merged_file",random_eff_model,"var_cope_file")


random_effects_analysis.connect(level2model,"design_mat",random_eff_model,"design_file")   
random_effects_analysis.connect(level2model,"design_con",random_eff_model,"t_con_file") 
random_effects_analysis.connect(level2model,"design_grp",random_eff_model,"cov_split_file") 

random_effects_analysis.connect(random_eff_model,"zstats",smooth_estimate,"zstat_file")

random_effects_analysis.connect(random_eff_model,"zstats",cluster_corr,"in_file") 
random_effects_analysis.connect(smooth_estimate,"dlh",cluster_corr,"dlh") 
random_effects_analysis.connect(num_of_voxels,("out_stat",pick_first),cluster_corr,"volume") 

random_effects_analysis.connect(copemerge,"merged_file",randomise,"in_file")
random_effects_analysis.connect(level2model,"design_con",randomise,"tcon")
random_effects_analysis.connect(level2model,"design_mat",randomise,"design_mat")

In [9]:
random_effects_analysis.connect(selectfiles,"copes",datasink,"copes")
random_effects_analysis.connect(selectfiles,"varcopes",datasink,"varcopes")

random_effects_analysis.connect(copemerge,"merged_file",datasink,"merge.@merged_cope_file")
random_effects_analysis.connect(varcopemerge,"merged_file",datasink,"merge.@merged_varcope_file")

random_effects_analysis.connect(level2model,"design_mat",datasink,"model.@design_mat")

random_effects_analysis.connect(random_eff_model,"copes",datasink,"random_effects.@copes")
random_effects_analysis.connect(random_eff_model,"var_copes",datasink,"random_effects.@var_copes")
random_effects_analysis.connect(random_eff_model,"weights",datasink,"random_effects.@weights")
random_effects_analysis.connect(random_eff_model,"zstats",datasink,"random_effects.@zstats")
random_effects_analysis.connect(random_eff_model,"res4d",datasink,"random_effects.@res4d")
random_effects_analysis.connect(random_eff_model,"tstats",datasink,"random_effects.@tstats")
random_effects_analysis.connect(cluster_corr,"threshold_file",datasink,"cluster.@z_cluster_corrected")
random_effects_analysis.connect(cluster_corr,"pval_file",datasink,"cluster.@pval_file")
random_effects_analysis.connect(cluster_corr,"localmax_txt_file",datasink,"cluster.@localmax_txt_file")
random_effects_analysis.connect(cluster_corr,"index_file",datasink,"cluster.@index_file")
random_effects_analysis.connect(randomise,"t_corrected_p_files",datasink,"non_parametric.@corr_p_values")


In [10]:
random_effects_analysis.write_graph(graph2use='flat', format='png', simple_form=True,
                          dotfilename= os.path.join(output,"random_effects_analysis_workflow.dot"))

random_effects_analysis.run(plugin='MultiProc', plugin_args={'n_procs' : 1})

210815-21:43:37,936 nipype.workflow INFO:
	 Generated workflow graph: /media/paradeisios/UUI/fmri/flanker/output/third_level/random_effects_analysis_workflow.png (graph2use=flat, simple_form=True).
210815-21:43:37,954 nipype.workflow INFO:
	 Workflow random_effects_analysis settings: ['check', 'execution', 'logging', 'monitoring']
210815-21:43:38,20 nipype.workflow INFO:
	 Running in parallel.
210815-21:43:38,27 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 8 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 1/1.
210815-21:43:38,82 nipype.workflow INFO:
	 [Node] Setting-up "random_effects_analysis.selectfiles" in "/media/paradeisios/UUI/fmri/flanker/output/third_level/random_effects_analysis/_contrast_list_6/selectfiles".
210815-21:43:38,95 nipype.workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
210815-21:43:38,146 nipype.workflow INFO:
	 [Node] Finished "random_effects_analysis.selectfiles".
210815-21:43:40,30 nipype.workflow INFO:


210815-21:44:20,148 nipype.workflow INFO:
	 [Node] Setting-up "random_effects_analysis.random_eff_model" in "/media/paradeisios/UUI/fmri/flanker/output/third_level/random_effects_analysis/_contrast_list_6/random_eff_model".
210815-21:44:20,150 nipype.workflow INFO:
	 [Node] Outdated cache found for "random_effects_analysis.random_eff_model".
210815-21:44:20,172 nipype.workflow INFO:
	 [Node] Running "random_eff_model" ("nipype.interfaces.fsl.model.FLAMEO"), a CommandLine Interface with command:
flameo --copefile=/media/paradeisios/UUI/fmri/flanker/output/third_level/random_effects_analysis/_contrast_list_6/copemerge/cope1_merged.nii.gz --covsplitfile=/media/paradeisios/UUI/fmri/flanker/output/third_level/random_effects_analysis/l2model/design.grp --designfile=/media/paradeisios/UUI/fmri/flanker/output/third_level/random_effects_analysis/l2model/design.mat --ld=stats --maskfile=/home/paradeisios/fsl/data/standard/MNI152_T1_2mm_brain_mask.nii.gz --runmode=ols --tcontrastsfile=/media/para

210815-21:44:38,293 nipype.workflow INFO:
	 [Node] Finished "random_effects_analysis.smooth_estimate".
210815-21:44:40,85 nipype.workflow INFO:
	 [Job 27] Completed (random_effects_analysis.smooth_estimate).
210815-21:44:40,87 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 9 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 1/1.
210815-21:44:40,138 nipype.workflow INFO:
	 [Node] Outdated cache found for "random_effects_analysis.cluster_corr".
210815-21:44:40,141 nipype.workflow INFO:
	 [Node] Setting-up "random_effects_analysis.cluster_corr" in "/media/paradeisios/UUI/fmri/flanker/output/third_level/random_effects_analysis/_contrast_list_5/cluster_corr".
210815-21:44:40,143 nipype.workflow INFO:
	 [Node] Outdated cache found for "random_effects_analysis.cluster_corr".
210815-21:44:40,153 nipype.workflow INFO:
	 [Node] Running "cluster_corr" ("nipype.interfaces.fsl.model.Cluster"), a CommandLine Interface with command:
cluster --dlh=0.0751541000 --in=/media/paradeisios

210815-21:45:02,237 nipype.workflow INFO:
	 [Node] Setting-up "random_effects_analysis.datasink" in "/media/paradeisios/UUI/fmri/flanker/output/third_level/random_effects_analysis/_contrast_list_3/datasink".
210815-21:45:02,239 nipype.workflow INFO:
	 [Node] Outdated cache found for "random_effects_analysis.datasink".
210815-21:45:02,250 nipype.workflow INFO:
	 [Node] Running "datasink" ("nipype.interfaces.io.DataSink")
210815-21:45:02,840 nipype.workflow INFO:
	 [Node] Finished "random_effects_analysis.datasink".
210815-21:45:04,109 nipype.workflow INFO:
	 [Job 39] Completed (random_effects_analysis.datasink).
210815-21:45:04,114 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 4 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 1/1.
210815-21:45:04,165 nipype.workflow INFO:
	 [Job 40] Cached (random_effects_analysis.randomise).
210815-21:45:06,114 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 3 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 1/1.
